# Running benchmarks

`````{margin}
````{dropdown} Necessary imports
```{code-block} python
from colosseum import config
from colosseum.agent.agents.episodic import PSRLEpisodic, QLearningEpisodic
from colosseum.agent.agents.infinite_horizon import QLearningContinuous, UCRL2Continuous
from colosseum.agent.utils import sample_agent_gin_configs_file
from colosseum.benchmark.benchmark import ColosseumDefaultBenchmark
from colosseum.benchmark.run import instantiate_and_get_exp_instances_from_agents_and_benchmarks
from colosseum.experiment import ExperimentConfig
from colosseum.experiment.experiment_instances import run_experiment_instances

experiments_folder = "tutorial"
experiment_name = "bench_run"
config.set_experiments_folder(experiments_folder, experiment_name)
config.set_hyperopt_folder(experiments_folder, experiment_name)
config.enable_verbose_logging()

seed = 42
```
````
`````

In [1]:
from colosseum import config
from colosseum.agent.agents.episodic import PSRLEpisodic, QLearningEpisodic
from colosseum.agent.agents.infinite_horizon import QLearningContinuous, UCRL2Continuous
from colosseum.agent.utils import sample_agent_gin_configs_file
from colosseum.benchmark.benchmark import ColosseumDefaultBenchmark
from colosseum.benchmark.run import instantiate_and_get_exp_instances_from_agents_and_benchmarks
from colosseum.experiment import ExperimentConfig
from colosseum.experiment.experiment_instances import run_experiment_instances

experiments_folder = "tutorial"
experiment_name = "bench_run"
config.set_experiments_folder(experiments_folder, experiment_name)
config.set_hyperopt_folder(experiments_folder, experiment_name)
config.enable_verbose_logging()

seed = 42

2022-10-15 16:42:38.870795: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-10-15 16:42:38.953001: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


2022-10-15 16:42:39.310709: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory
2022-10-15 16:42:39.310749: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or directory
2022-10-15 16:42:39.310754: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Cannot dlopen some TensorRT libraries. If you would like to use Nvidia GPU with TensorRT, please make sure the missing libraries mentioned above are installed properly.


This short tutorial shows how to obtain agents' performances in a {{col}} benchmark using your local machine, whereas the [_Running on a cluster _ tutorial](../tutorials/benchmark-scaling) tutorial illustrates how to scale the benchmarking procedure to a generic cluster.

```{code-block} python
# Define a small scale experiment config
experiment_config = ExperimentConfig(
    n_seeds=1,
    n_steps=5_000,
    max_interaction_time_s=1 * 30,
    log_performance_indicators_every=1000,
)

# Take the default colosseum benchmark for the episodic ergodic and the continuous communicating settings
b_e = ColosseumDefaultBenchmark.EPISODIC_QUICK_TEST.get_benchmark()
b_e.experiment_config = experiment_config
b_c = ColosseumDefaultBenchmark.CONTINUOUS_QUICK_TEST.get_benchmark()
b_c.experiment_config = experiment_config

# Randomly sample some episodic agents
agents_configs_e = {
    PSRLEpisodic : sample_agent_gin_configs_file(PSRLEpisodic, n=1, seed=seed),
    QLearningEpisodic : sample_agent_gin_configs_file(QLearningEpisodic, n=1, seed=seed),
}

# Randomly sample some continuous agents
agents_configs_c = {
    QLearningContinuous : sample_agent_gin_configs_file(QLearningContinuous, n=1, seed=seed),
    UCRL2Continuous : sample_agent_gin_configs_file(UCRL2Continuous, n=1, seed=seed),
}

# Obtain the experiment instances for the agents configurations and the benchmark
agents_and_benchmarks = [
    (agents_configs_e, b_e),
    (agents_configs_c, b_c),
]
experiment_instances = instantiate_and_get_exp_instances_from_agents_and_benchmarks(agents_and_benchmarks)

# Run the experiment instances
# Note that if multiprocessing is enabled, Colosseum will take advantage of it
run_experiment_instances(experiment_instances)
```

In [2]:
import shutil
shutil.rmtree(config.get_experiments_folder())